# Train model with noisy signal

This notebook is based on Mauri AG1LE's work back in 2015 that can be found on Github [here](https://github.com/ag1le/RNN-Morse). From an audio signal generated at 8 kHz sample rate (thus in 4 kHz bandwidth) it attempts to recognize Morse code distinctive features in the signal that is:

  - The envelope
  - The "dit"
  - The "dah"
  - The element separator at the end of a "dit" or a "dah"
  - The character separator
  - The word separator
  
It trains a LSTM based recurrent neural network (RNN) on a slightly noisy signal (a few dB SNR in 4 kHz bandwidth) in an encoder-decoder fashion. The envelope of the signal is taken as input as a time series of floating point values and the labels are also time series of the 6 signals described above.

It then attempts prediction on a much noisier signal of the same test data to see how it can perform in retrieving the 6 predicted signals and reformat the original envelope.

This is the Keras / Tensorflow version.

In [ ]:
!pip install sounddevice

In [ ]:
!sudo apt-get install libportaudio2

## Generate annotated raw signal

In [ ]:
import MorseGen
import matplotlib.pyplot as plt 

#phrase = '01234 6789 QUICK BROWN FOX 01234 6789 QUICK BROWN FOX01234 6789 QUICK BROWN FOX01234 6789 QUICK BROWN FOX01234 6789 QUICK BROWN FOX 01234 6789 QUICK BROWN FOX'
#phrase = '7U7K 0DC55B H ZN0J Q9 H2X0 LZ16A ECA2DE 6A2 NUPU 67IL6EIH YVZA 5OTGC3U C3R PGW RS0 84QTV4PB EZ1 JBGJ TT1W4M5PBJ GZVLWXQG 7POU6 FMTXA N3CZ Y1Q9VZ6 9TVL CWP8KSB'
phrase = '6 WREB W7UU QNWXS2 3KRO72Q AN1TI QZIWH G L0U7 Y17X45 OVIC2 C052W00PI60 O5Y 10R2N 4 FHC JXRGS4 DWBOL7ZUXJU EMNC3 WWBNT7 0UP GMKQ YG83H8 IT2Q Y0YBZ SQ80I5 W7SW 0K BMJ8JPM 51CK1 R08T 7SU1LYS7W6T 4JKVQF V3G UU2O1OM4 P4B 4A9DLC VI1H 4 HMP57 Q6G3 4QADIG FRJ 0MVL EPSM CS N9IZEMA GSRWUPBYB FD29 YI3PY N31W X88NS 773EW4Q4 LSW'
Fs = 8000
morse_gen = MorseGen.Morse()
samples_per_dit = morse_gen.nb_samples_per_dit(Fs, 13)
label_df = morse_gen.encode_df(phrase, samples_per_dit)
print(label_df.shape)
plt.figure(figsize=(50,5))
x = 200000
y = 300000
plt.plot(label_df[x:y].env*0.9 + 0.0, label='env')
plt.plot(label_df[x:y].dit*0.9 + 1.0, label='dit')
plt.plot(label_df[x:y].dah*0.9 + 2.0, label='dah')
plt.plot(label_df[x:y].ele*0.9 + 3.0, label='ele')
plt.plot(label_df[x:y].chr*0.9 + 4.0, label='chr')
plt.plot(label_df[x:y].wrd*0.9 + 5.0, label='wrd')
plt.title("labels")
plt.legend()
plt.grid()

### Audio

In [ ]:
import numpy as np

Fc = 600
SNR_dB = 6
SNR_linear = 10.0**(SNR_dB/10.0)
t = np.linspace(0, len(label_df)-1, len(label_df))
cw = np.sin((Fc/Fs)*2*np.pi*t)
morsecode = cw * label_df.env
power = morsecode.var()
noise_power = power/SNR_linear
noise = np.sqrt(noise_power)*np.random.normal(0, 1, len(morsecode))
signal = morsecode + noise

plt.figure(figsize=[25,5])
plt.plot(signal[200000:300000])

### Labels

In [ ]:
plt.figure(figsize=(25,5))
x = 200000
y = 300000
plt.plot(label_df[x:y].ele*0.07 + 0.1)
plt.plot(label_df[x:y].chr*0.07 + 0.2)
plt.plot(label_df[x:y].wrd*0.07 + 0.3)
plt.plot(label_df[x:y].dit*0.07 + 0.4)
plt.plot(label_df[x:y].dah*0.07 + 0.5)
plt.plot((label_df[x:y].dit+label_df[x:y].dah))
plt.title("labels")

### Plot spectrogram

In [ ]:
plt.figure(figsize=(25,10))
plt.ylabel("Frequency (Hz)")
plt.xlabel("Time (seconds)")
_ = plt.specgram(signal[0:30*Fs], NFFT=256, Fs=Fs, Fc=0, mode='magnitude', noverlap=128)

## Generate final annotated data
### Find peak

In [ ]:
import MorseDSP

maxtab, f, s = MorseDSP.find_peak(Fs, signal)
tone = maxtab[0,0]
plt.title("Morse signal peak found at {} Hz".format(tone))
plt.xlabel("Frequency (Hz)")
plt.ylabel("Amplitude (log)")
plt.yscale('log')
_ = plt.plot(f[0:int(len(f)/2-1)], abs(s[0:int(len(s)/2-1)]),'g-')
_ = plt.scatter(maxtab[:,0], maxtab[:,1], c='r') 
plt.show()

### Generate image

In [ ]:
nside_bins = 1
nfft = 256
f, t, img, noverlap = MorseDSP.specimg(Fs, signal, None, None, tone, nfft, nside_bins)
decim = nfft - noverlap
print(type(signal), signal.shape)
print(type(f), f.shape)
print(type(t), t.shape, max(t))
print(type(img), img.shape)
print(noverlap, len(signal)//noverlap, decim)
# Show first 25 seconds at most
rmax = 25 / max(t) if max(t) > 25 else 25
imax = int(rmax*len(t))
t1 = t[:imax]
img1 = img[:,:imax]
plt.figure(figsize=(30,3))
plt.pcolormesh(t1, f, img1, shading='flat', cmap=plt.get_cmap('binary'))
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()  

### Generate spectral line

In [ ]:
plt.figure(figsize=(30,3))
plt.plot(img[nside_bins-1][:1500], label="-1")
plt.plot(img[nside_bins][:1500], label="0")
plt.plot(img[nside_bins+1][:1500], label="+1")
plt.legend()
plt.show()  

In [ ]:
import numpy as np

img_line = np.sum(img, axis=0)
img_line /= max(img_line)
print(img_line.shape)
plt.figure(figsize=(30,3))
plt.plot(img_line[:1500], label="lin")
plt.legend()
plt.show()

### Generate training dataset

In [ ]:
label_img = label_df[::decim] # decimate labels by spectrum decimation
label_img.reset_index(drop=True, inplace=True)
#img_sig = img[nside_bins, :] # take line at center bin which is next to the low side bins
img_sig = label_img.dit + label_img.dah # ideal signal
img_sig /= max(img_sig)                 # normalize
train_img = label_img
train_img = train_img.truncate(after=len(img_line)-1, copy=False)
x = 0
y = imax
print(x, y)
plt.figure(figsize=(30,5))
plt.plot(train_img[x:y].wrd + 0.0, label="wrd")
plt.plot(train_img[x:y].chr + 1.0, label="chr")
plt.plot(train_img[x:y].ele + 2.0, label="ele")
plt.plot(train_img[x:y].dit + 3.0, label="dit")
plt.plot(train_img[x:y].dah + 4.0, label="dah")
plt.plot(train_img[x:y].env + 5.0, label="env")
plt.plot(img_line[x:y] + 6.0, label="lin")
plt.title("image line and labels")
plt.grid()
plt.legend()
print(train_img.shape, img_line.shape, type(img_sig))

### Split training and test data

In [ ]:
import numpy as np

def _load_data(data_x, data_y, n_prev):  
    """
    data_y should be pd.DataFrame()
    """

    docX, docY = [], []
    for i in range(len(data_y)-n_prev):
        docX.append(data_x[i:i+n_prev]) # sliding n_prev values
        docY.append(data_y.iloc[i+n_prev].values)   # next value from previous n_prev values
    alsX = np.array(docX)
    alsY = np.array(docY)

    return alsX, alsY

def train_test_split(x, label_df, n_prev=100, test_size=0.5):  
    """
    This just splits data to training and testing parts
    """
    ntrn = round(len(label_df) * (1 - test_size))

    X_train, y_train = _load_data(x[0:ntrn], label_df.iloc[0:ntrn], n_prev)
    X_test, y_test = _load_data(x[ntrn:], label_df.iloc[ntrn:], n_prev)

    return ntrn, (X_train, y_train), (X_test, y_test)

In [ ]:
n_prev = 200
X_img = img_line.reshape((len(img_line), 1))
ntrn, (X_train, y_train), (X_test, y_test) = train_test_split(X_img, train_img, n_prev)  # retrieve data
l_train = img_line[0:ntrn+n_prev]
l_test = img_line[ntrn+n_prev:]
print (ntrn, img_sig.shape, train_img.shape, X_train.shape, y_train.shape, l_train.shape, l_test.shape)
a = []
b = []
for t in range(5):
    a.append(X_test[t*n_prev])
    b.append(X_train[t*n_prev])
plt.figure(figsize=(25,3))
plt.plot(np.concatenate((tuple(a)))*0.5, label='test')
plt.plot(np.concatenate((tuple(b)))*0.5+0.5, label='train')
plt.title("Train and test")
plt.legend()

In [ ]:
print(X_train[:,:,0].shape)
a = []
for i in range(5):
    a.append(X_test[:,:,0][i*n_prev])
plt.figure(figsize=(25,3))
plt.plot(np.concatenate(tuple(a)), label='X_test')
plt.plot(l_test[:5*n_prev]+1.0, label='line')
plt.plot(y_test[:5*n_prev,0]+2.0, label='y_test')
plt.title("Test")
plt.legend()

## Create model

In [ ]:
from keras.models import Sequential  
from keras.layers.core import Dense, Activation  
from keras.layers.recurrent import LSTM
from keras.callbacks import Callback
from keras.layers import Bidirectional

in_neurons = 1  
out_neurons = 6  
hidden_neurons = 12

lstm = LSTM(hidden_neurons, input_shape=(n_prev,in_neurons), return_sequences=False)
model = Sequential()  
model.add(Bidirectional(lstm, merge_mode='sum', input_shape=(n_prev,in_neurons))) 
#model.add(LSTM(hidden_neurons, input_dim=in_neurons, return_sequences=False)) 
model.add(Dense(out_neurons, input_dim=hidden_neurons))
model.add(Activation("linear"))  
model.compile(loss="mean_squared_error", optimizer="rmsprop")  

# Call back to capture losses 
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
# Train the model
history = LossHistory()
model.summary()

## Train model

In [ ]:
X_train[0].shape, y_train[0].shape, X_test.shape

In [ ]:
hist = model.fit(X_train, y_train, batch_size=400, epochs=60, validation_split=0.05, callbacks=[history])  
print (hist.history)
plt.figure(figsize=(20,3))
plt.plot( history.losses)
plt.title("training loss")

In [ ]:
predicted = model.predict(X_test)
rmse = np.sqrt(((predicted - y_test) ** 2).mean(axis=0))
print(rmse)

## Predict (test)

In [ ]:
plt.figure(figsize=(30,3))
plt.plot(y_test)

In [ ]:
plt.figure(figsize=(25,3))
lines = plt.plot(y_test[:y])
plt.title("actual categories")
labels=['env','dit','dah','elt','chr','wrd']
plt.legend(lines, labels)
print(X_test.shape, y_test.shape)

In [ ]:
plt.figure(figsize=(30,7))
plt.plot(y_test[:y,0], label="y0")
plt.plot(l_test[:y] + 1.0, label="lin")
plt.plot(predicted[:y,0] + 2.0, label="sig")
plt.plot(predicted[:y,1] + 3.0, label="dit")
plt.plot(predicted[:y,2] + 4.0, label="dah")
plt.title("Predictions")
plt.legend()
plt.grid()

In [ ]:
sig = predicted[:y,0]
sig = (sig - min(sig)) / (max(sig) - min(sig))
mod = predicted[:y,1] + predicted[:y,2] - 3.0*(predicted[:y,3] + predicted[:y,4] + predicted[:y,5])
mod = (mod - min(mod)) / (max(mod) - min(mod))
mor = y_test[:y,1] + y_test[:y,2] - 3.0*(y_test[:y,3] + y_test[:y,4] + y_test[:y,5])
mor = (mor - min(mor)) / (max(mor) - min(mor))
plt.figure(figsize=(30,5))
plt.plot(sig, label="sig")
plt.title("predicted signal modulation")
plt.grid()
plt.figure(figsize=(30,5))
plt.plot(mod*0.6+0.4, label="mod")
plt.plot(mor*0.3, label="mor")
plt.plot(l_test[:y]*0.3, label="sig")
plt.title("reconstructed signal modulation with 'dah' and 'dit'")
plt.legend()
plt.grid()

In [ ]:
mor = y_test[:y,1] + y_test[:y,2]
plt.figure(figsize=(25,3))
plt.plot(predicted[:y,1], label='dit')
plt.plot(predicted[:y,2], label='dah')
plt.plot(mor*0.5 + 1.0, label='mor')
plt.title("'dit' and 'dah' symbols prediction vs modulation")
plt.legend()
plt.grid()

In [ ]:
plt.figure(figsize=(25,3))
plt.plot(predicted[:y,3], label='ele')
plt.plot(mor, label='mor')
plt.title("Element space prediction vs modulation")
plt.legend()

In [ ]:
plt.figure(figsize=(25,3))
plt.plot(predicted[:y,4] ,label='chr')
plt.plot(mor, label='mor')
plt.title("Character space prediction vs modulation")
plt.legend()

In [ ]:
plt.figure(figsize=(25,3))
plt.plot(predicted[:y,5], label='wrd')
plt.plot(mor, label='mor')
plt.title("Word space prediction vs modulation")
plt.legend()

In [ ]:
plt.figure(figsize=(25,5))
plt.plot(predicted[:y,1], label="dit")
plt.plot(predicted[:y,2] + 1.0, label="dah")
plt.plot(predicted[:y,3] + 2.0, label="ele")
plt.plot(predicted[:y,4] + 3.0, label="chr")
plt.plot(predicted[:y,5] + 4.0, label="wrd")
plt.plot(l_test[:y] + 5.0, label="sig")
plt.plot(mor*6.0, label="mor")
plt.title("Altogether vs signal and modulation")
plt.legend()

In [ ]:
from scipy.io import wavfile

Fcode = 600
Fs = 8000
emod = np.array([np.exp(x) if x > 0.5 else 0.0 for x in mod])
emod /= max(emod)
remod = np.array([[x]*noverlap for x in emod]).flatten()
wt = (Fcode / Fs)*2*np.pi
tone = np.sin(np.arange(len(remod))*wt)
wavfile.write('audio/re.wav', Fs, tone*remod)
plt.figure(figsize=(25,5))
plt.plot(tone*remod)
plt.title("reconstructed signal")
plt.grid()
# .4QTV4PB EZ1 JBGJ TT1W4M...
# 7U7K 0DC55B H ZN0J Q9 H2X0 LZ16A ECA2DE 6A2 NUPU 67IL6EIH YVZA 5OTGC3U C3R PGW RS0 84QTV4PB EZ1 JBGJ TT1W4M5PBJ GZVLWXQG 7POU6 FMTXA N3CZ Y1Q9VZ6 9TVL CWP8KSB'

In [ ]:
omod = l_test[:y]
omod / max(omod)
orig_mod = np.array([[x]*decim for x in omod]).flatten()
wavfile.write('audio/or.wav', Fs, tone*orig_mod)
plt.figure(figsize=(25,5))
plt.plot(tone*orig_mod)
plt.title("original filtered signal")
plt.grid()

## Make new predictions

In [ ]:
SNR_dB = -13
SNR_linear = 10.0**(SNR_dB/10.0)
noise_power = power/SNR_linear
noise = np.sqrt(noise_power)*np.random.normal(0, 1, len(morsecode))
signal1 = morsecode + noise

### Find peak

In [ ]:
maxtab, f, s = MorseDSP.find_peak(Fs, signal1)
tone = maxtab[0,0]
plt.title("Morse signal peak found at {} Hz".format(tone))
plt.xlabel("Frequency (Hz)")
plt.ylabel("Amplitude (log)")
plt.yscale('log')
_ = plt.plot(f[0:int(len(f)/2-1)], abs(s[0:int(len(s)/2-1)]),'g-')
_ = plt.scatter(maxtab[:,0], maxtab[:,1], c='r') 
plt.show()

### Generate image

In [ ]:
nside_bins = 1
nfft = 256
f, t, img, noverlap = MorseDSP.specimg(Fs, signal1, None, None, tone, nfft, nside_bins)
decim = nfft - noverlap
print(type(signal1), signal1.shape)
print(type(f), f.shape)
print(type(t), t.shape, max(t))
print(type(img), img.shape)
print(noverlap, len(signal1)//noverlap, decim)
# Show first 25 seconds at most
rmax = 25 / max(t) if max(t) > 25 else 25
imax = int(rmax*len(t))
t1 = t[:imax]
img1 = img[:,:imax]
plt.figure(figsize=(30,3))
plt.pcolormesh(t1, f, img1, shading='flat', cmap=plt.get_cmap('binary'))
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()  

### Generate spectral line

In [ ]:
plt.figure(figsize=(30,3))
plt.plot(img[nside_bins-1][:1500], label="-1")
plt.plot(img[nside_bins][:1500], label="0")
plt.plot(img[nside_bins+1][:1500], label="+1")
plt.legend()
plt.show()

In [ ]:
img_line = img[nside_bins] #np.sum(img, axis=0)
img_line /= max(img_line)
print(img_line.shape)
plt.figure(figsize=(30,3))
plt.plot(img_line[:1500], label="lin")
plt.legend()
plt.show()

### Same dataset

In [ ]:
plt.figure(figsize=(30,5))
plt.plot(train_img[x:y].wrd + 0.0, label="wrd")
plt.plot(train_img[x:y].chr + 1.0, label="chr")
plt.plot(train_img[x:y].ele + 2.0, label="ele")
plt.plot(train_img[x:y].dit + 3.0, label="dit")
plt.plot(train_img[x:y].dah + 4.0, label="dah")
plt.plot(train_img[x:y].env + 5.0, label="env")
plt.plot(img_line[x:y] + 6.0, label="lin")
plt.title("image line and labels")
plt.grid()
plt.legend()
print(train_img.shape, img_line.shape, type(img_sig))

### Get test data

In [ ]:
X_img = img_line.reshape((len(img_line), 1))
ntrn, (X_train, y_train), (X_test, y_test) = train_test_split(X_img, train_img, n_prev)  # retrieve data
l_train = img_line[0:ntrn+n_prev]
l_test = img_line[ntrn+n_prev:]
print (ntrn, img_sig.shape, train_img.shape, X_train.shape, y_train.shape, l_train.shape, l_test.shape)
a = []
b = []
for t in range(5):
    a.append(X_test[t*n_prev])
    b.append(X_train[t*n_prev])
plt.figure(figsize=(25,3))
plt.plot(np.concatenate((tuple(a)))*0.5, label='test')
plt.plot(np.concatenate((tuple(b)))*0.5+0.5, label='train')
plt.title("Train and test")
plt.legend()

In [ ]:
a = []
for i in range(5):
    a.append(X_test[:,:,0][i*n_prev])
plt.figure(figsize=(25,3))
plt.plot(np.concatenate(tuple(a)), label='test')
plt.plot(l_test[:5*n_prev]+1.0, label='line')
plt.title("Test")
plt.legend()

In [ ]:
print (img_line.shape, X_test.shape)

In [ ]:
predicted1 = model.predict(X_test)
rmse1 = np.sqrt(((predicted1 - y_test) ** 2).mean(axis=0))
print(rmse1)

In [ ]:
ori1 = l_test[:y]
ori1 = (ori1 - min(ori1)) / (max(ori1) - min(ori1))
sig1 = predicted1[:y,0]
sig1 = (sig1 - min(sig1)) / (max(sig1) - min(sig1))
dit1 = predicted1[:y,1]
dit1 = (dit1 - min(dit1)) / (max(dit1) - min(dit1))
dah1 = predicted1[:y,2]
dah1 = (dah1 - min(dah1)) / (max(dah1) - min(dah1))
dahdit1 = (dit1 + dah1) / 2.0
sil1 = predicted1[:y,3] + predicted1[:y,4] + predicted1[:y,5]
mod1 = predicted1[:y,0] - sil1
mod1 = (mod1 - min(mod1)) / (max(mod1) - min(mod1))
plt.figure(figsize=(50,8))
plt.plot(ori1, label="ori")
plt.plot(sig1 + 1.0, label="sig")
plt.plot(mod1 + 2.0, label="mod")
plt.plot(dahdit1 + 3.0, label="dd")
plt.plot(sil1 + 4.0, label="sil")
plt.plot(predicted1[:y,3] + 5.0, label='ele')
plt.plot(predicted1[:y,4] + 6.0, label='chr')
plt.plot(predicted1[:y,5] + 7.0, label='wrd')
plt.plot((y_test[:y,1]+y_test[:y,2])*8.2, label="mor")
# plt.plot(dit1*0.3, label="dit")
# plt.plot(dah1*0.3, label="dah")
plt.title("predictions")
plt.legend()
plt.grid()
plt.figure(figsize=(50,5))
plt.plot(mod1, label="mod")
plt.plot(sig1+1.0, label="sig")
plt.plot((y_test[:y,1]+y_test[:y,2])*2.0, label="mor")
plt.title("reconstructed signal modulation with 'dah' and 'dit'")
plt.legend()
plt.grid()

In [ ]:
plt.figure(figsize=(50,10))
plt.plot(predicted1[:y,1], label="dit")
plt.plot(predicted1[:y,2] + 1.0, label="dah")
plt.plot(predicted1[:y,3] + 2.0, label="ele")
plt.plot(predicted1[:y,4] + 3.0, label="chr")
plt.plot(predicted1[:y,5] + 4.0, label="wrd")
plt.plot(predicted1[:y,0] + 5.0, label="sig")
plt.plot(l_test[:y] + 6.0, label="inp")
plt.plot(y_test[:y,0]*7.0, label="mod")
plt.title("Altogether vs signal and modulation")
plt.legend()
plt.grid()

In [ ]:
import scipy as sp

#omod = np.array([sp.special.expit(12*(x-0.3)) for x in l_test[:y]])
omod = np.array([sp.special.expit(20*(x-0.18)) for x in l_test[:y]])
orig_mod = np.array([[x]*decim for x in omod]).flatten()
orig_mod /= max(orig_mod)
wt = (Fcode / Fs)*2*np.pi
tone = np.sin(np.arange(len(orig_mod))*wt)
wavfile.write('audio/or1.wav', Fs, tone*orig_mod)
ref_mod = np.array([[x]*decim for x in mor]).flatten()
plt.figure(figsize=(25,5))
plt.plot(tone*orig_mod, label='mod')
plt.plot(ref_mod*1.2, label='mor')
plt.title("original filtered signal")
plt.legend()
plt.grid()

In [ ]:
import scipy as sp

def modscale(x):
    return sp.special.expit(20*(x-0.28))
    
emod = np.array([modscale(x) for x in mod1])
emod /= max(emod)
#emod = modn
remod = np.array([[x]*decim for x in emod]).flatten()
remor = np.array([[x]*decim for x in mor]).flatten()
wt = (Fcode / Fs)*2*np.pi
tone = np.sin(np.arange(len(remod))*wt)
wavfile.write('audio/re1.wav', Fs, tone*remod)
plt.figure(figsize=(25,5))
plt.plot(tone*remod, label='filt')
plt.plot(remor*1.2, label='omod')
plt.title("reconstructed signal")
plt.legend()
plt.grid()

In [ ]:
import scipy as sp

sx = np.linspace(0, 1, 121)
sy = sp.special.expit(20*(sx-0.25))
plt.plot(sx, sy)
plt.grid()
plt.xlabel('x')
plt.title('expit(x)')
plt.show()